In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_excel('data_training_nbc_pix.xlsx', header=0)
df.head()

,No,from_user,text,kelas,label
0,1,voaindonesia,negara negara kawasan asia senang serang alas ...,kontra,-1
1,2,dobbybunni,blackwafferroll jeonqjaehyvn 2cool4skuy jntexp...,pro,1
2,3,DoniMardani_,puan kondisi global akibat perang rusia ukrain...,kontra,-1
3,4,DoniMardani_,ketua dpr ri puan maharani perintah antisipasi...,kontra,-1
4,5,KemotThe,putinmoskva1 ekonomi rusia stabil kuat tingkat...,kontra,-1


In [3]:
def clean_lower(lwr):
    lwr = lwr.lower()
    return lwr

df['lwr'] = df['text'].apply(clean_lower)
casefolding = pd.DataFrame(df['lwr'])
casefolding

,lwr
0,negara negara kawasan asia senang serang alas ...
1,blackwafferroll jeonqjaehyvn 2cool4skuy jntexp...
2,puan kondisi global akibat perang rusia ukrain...
3,ketua dpr ri puan maharani perintah antisipasi...
4,putinmoskva1 ekonomi rusia stabil kuat tingkat...
...,...
583,bunda rose15 rangsundatea agussup39011241 yrad...
584,juanmahaganti adyscore s detikcom bang serius ...
585,perang gila rusia duduk 42 kota desa ukraina t...
586,softwarnews ukraina usd 7 miliar rekonstruksi ...


In [4]:
#Remove Puncutuation
clean_spcl = re.compile('[/(){}\[\]\|,;]')
clean_symbol = re.compile('[^0-9a-z]')
def clean_punct(text):
    text = clean_spcl.sub('', text)
    text = clean_symbol.sub(' ', text)
    return text# Buat kolom tambahan untuk data description yang telah diremovepunctuation   
df['clean_punct'] = df['lwr'].apply(clean_punct)
df['clean_punct']

0      negara negara kawasan asia senang serang alas ...
1      blackwafferroll jeonqjaehyvn 2cool4skuy jntexp...
2      puan kondisi global akibat perang rusia ukrain...
3      ketua dpr ri puan maharani perintah antisipasi...
4      putinmoskva1 ekonomi rusia stabil kuat tingkat...
                             ...                        
583    bunda rose15 rangsundatea agussup39011241 yrad...
584    juanmahaganti adyscore s detikcom bang serius ...
585    perang gila rusia duduk 42 kota desa ukraina t...
586    softwarnews ukraina usd 7 miliar rekonstruksi ...
587    stevaniehuangg nilai kurs negara gantung perca...
Name: clean_punct, Length: 588, dtype: object

In [5]:
def _normalize_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")
df['clean_double_ws'] = df['clean_punct'].apply(_normalize_whitespace)
df['clean_double_ws']

0      negara negara kawasan asia senang serang alas ...
1      blackwafferroll jeonqjaehyvn 2cool4skuy jntexp...
2      puan kondisi global akibat perang rusia ukrain...
3      ketua dpr ri puan maharani perintah antisipasi...
4      putinmoskva1 ekonomi rusia stabil kuat tingkat...
                             ...                        
583    bunda rose15 rangsundatea agussup39011241 yrad...
584    juanmahaganti adyscore s detikcom bang serius ...
585    perang gila rusia duduk 42 kota desa ukraina t...
586    softwarnews ukraina usd 7 miliar rekonstruksi ...
587    stevaniehuangg nilai kurs negara gantung perca...
Name: clean_double_ws, Length: 588, dtype: object

In [6]:
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah', 'https'])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
#txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
#list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
#def stopwords_removal(words):
 #   return [word for word in words if word not in list_stopwords]

def clean_stopwords(text):
    text = ' '.join(word for word in text.split() if word not in list_stopwords) # hapus stopword dari kolom deskripsi
    return text# Buat kolom tambahan untuk data description yang telah distopwordsremoval   

df['clean_sw'] = df['clean_double_ws'].apply(clean_stopwords)
df['clean_sw']

0      negara negara kawasan asia senang serang alas ...
1      blackwafferroll jeonqjaehyvn 2cool4skuy jntexp...
2      puan kondisi global akibat perang rusia ukrain...
3      ketua dpr ri puan maharani perintah antisipasi...
4      putinmoskva1 ekonomi rusia stabil kuat tingkat...
                             ...                        
583    bunda rose15 rangsundatea agussup39011241 yrad...
584    juanmahaganti adyscore s detikcom bang serius ...
585    perang gila rusia duduk 42 kota desa ukraina t...
586    softwarnews ukraina usd 7 miliar rekonstruksi ...
587    stevaniehuangg nilai kurs negara gantung perca...
Name: clean_sw, Length: 588, dtype: object

In [7]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

df['sudah_bersih'] = df['clean_sw'].apply(stemmer.stem)
df['sudah_bersih']

0      negara negara kawasan asia senang serang alas ...
1      blackwafferroll jeonqjaehyvn 2cool4skuy jntexp...
2      puan kondisi global akibat perang rusia ukrain...
3      ketua dpr ri puan maharani perintah antisipasi...
4      putinmoskva1 ekonomi rusia stabil kuat tingkat...
                             ...                        
583    bunda rose15 rangsundatea agussup39011241 yrad...
584    juanmahaganti adyscore s detikcom bang serius ...
585    perang gila rusia duduk 42 kota desa ukraina t...
586    softwarnews ukraina usd 7 miliar rekonstruksi ...
587    stevaniehuangg nilai kurs negara gantung perca...
Name: sudah_bersih, Length: 588, dtype: object

In [8]:
df.to_excel("token nbc tugas akhir.xlsx")